In [ ]:
from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import time
import re

conn = sqlite3.connect('inven.db')
cur = conn.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS inven;
    CREATE TABLE inven(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        url TEXT NOT NULL,
        head TEXT NOT NULL,
        body TEXT NOT NULL,
        comment TEXT NOT NULL,
        date TEXT NOT NULL
    );
''')

In [ ]:
url = 'http://www.inven.co.kr/board/lol/4625'
session = Session()

urls = list()
seen = list()

urls.append(url)

count = 0

while urls:
    
    try:
        count += 1
        seed = urls.pop(0)

        resp = session.request('GET', seed)
        dom = BeautifulSoup(resp.text, 'lxml')
        time.sleep(0.02)

        seen.append(seed)
        

        # 이 부분이 목록에서만 실행되도록 조건문 넣어줘야함.
        # 그리고 그 안에 next라는 이미지 찾으면 page에 append 되게 조건 넣어줘야함. 
        for _ in [_['href'] for _ in dom.select('tr.tr > td.bbsSubject > a')
                  if _.has_attr('href')\
                 and re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+', _['href'])]:
            newUrls = urljoin(seed, _)
            if newUrls not in urls \
                    and newUrls not in seen:
                urls.append(newUrls)
                
        if dom.select_one('div.articleContent #powerbbsContent') != None:
            head = dom.select_one('div.articleTitle > h1').text.strip()
            body = dom.select_one('div.articleContent #powerbbsContent').text.strip()
            commenturl = urljoin(url, '/common/board/comment.json.php')
            data = {
                'comeidx': '4625',
                'articlecode': '',
                'sortorder': 'date',
                'act': 'list',
                'out': 'json',
                'replynick':'', 
                'replyidx': '0',
                'uploadurl':'', 
                'imageposition': '',
            }
            data['articlecode'] = re.search(r'/(\d+$)', seed).group(1)
            commentresp = session.request('POST', commenturl, data = data)
            result = commentresp.json()
            comment = [_['o_comment'] for _ in result['commentlist'][0]['list']]
            comment = [_.split('&amp;nbsp;') for _ in comment]
            comment = [''.join(_) for _ in comment]
            date = dom.select_one('.articleInfo > articleInfo').text.strip()
            cur.execute('''
                INSERT INTO instiz(url, head, body, comment, date) VALUES(?,?,?,?,?)
            ''', [seed, head, body, comments, date])
            conn.commit()

        if count % 100 == 0:
            print(count)
    except:
        print('Error')

In [42]:
url = 'http://www.inven.co.kr/board/lol/4625/2705057'
session = Session()
resp = session.request('GET', url, headers=headers)
dom = BeautifulSoup(resp.text, 'html.parser')

commenturl = urljoin(url, '/common/board/comment.json.php')
data = {
    'comeidx': '4625',
    'articlecode': '',
    'sortorder': 'date',
    'act': 'list',
    'out': 'json',
    'replynick':'', 
    'replyidx': '0',
    'uploadurl':'', 
    'imageposition': '',
}
data['articlecode'] = re.search(r'/(\d+$)', url).group(1)
commentresp = session.request('POST', commenturl, data = data)
result = commentresp.json()
comment = [_['o_comment'] for _ in result['commentlist'][0]['list']]
comment = [_.split('&amp;nbsp;') for _ in comment]
comment = [''.join(_) for _ in comment]

In [ ]:
# DC 댓글 가져오기
resp = session.request('GET','https://gall.dcinside.com/board/view/?id=ib_new1&no=7553183&_rk=eYK&page=1')
url = 'https://gall.dcinside.com/board/comment/'
data = {
    'id':'ib_new1',
    'no':'',
    'cmt_id':'ib_new1',
    'cmt_no':'',
    'e_s_n_o':'e_s_n_o',
    'comment_page':'1',

}
data['cmt_no'] = data['no'] = re.search(r'no=([^&]+)',urlparse(resp.url)[4]).group(1)
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36', 'referer':resp.url, 'X-Requested-With': 'XMLHttpRequest'}
resp = session.request('POST',url,headers= headers, data=data)
result = resp.json()